# Testing types of hyperparameters

In [4]:
import numpy as np
import pandas as pd
import os

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier

In [5]:
# Get the paths
env_path = '/data/dssg-disinfo/.env'
load_dotenv(env_path, override=True)
DATA_PATH = os.getenv("PATH") # we need to change "PATH" to "DATA_PATH" in the ENV File 
CLEAN_DATA = os.getenv("CLEAN_DATA")
df = pd.read_csv(os.path.join(DATA_PATH, CLEAN_DATA))

### III. Splitting the data into training and testing
X = df['article_text'] # article_text
y = df.label

NameError: name 'load_dotenv' is not defined

In [6]:
import baseline
from baseline import register_model_arch,register_model_arch, _model_arch_registry
from baseline import build_model_arch, register_model_arch, build_model, get_data_and_split
from baseline import compile_model

In [7]:
model=build_model()
compiled_model= compile_model(model)

Model summary:
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 681, 300)          3000000   
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               186880    
_________________________________________________________________
dense (Dense)                (None, 64)                8256      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 3,195,201
Trainable params: 3,195,201
Non-trainable params: 0
_________________________________________________________________


# Function to create model, required for KerasClassifier
def create_model(optimizer='adam'):
	# create model
	model = Sequential()
	model.add(Dense(12, input_dim=8, activation='relu'))
	model.add(Dense(1, activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
	return model

In [9]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)
# load dataset
#dataset = np.loadtxt("pima-indians-diabetes.csv", delimiter=",")
# split into input (X) and output (Y) variables
#X = dataset[:,0:8]
#Y = dataset[:,8]
# create model
classifer_model = KerasClassifier(build_fn=compiled_model, epochs=100, batch_size=10, verbose=0)
# define the grid search parameters
optimizer = ['adam'] #['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
batch_size = [10,20] #[10, 20, 40, 60, 80, 100]
epochs = [2, 3] #[2,5,7,10, 50, 100]
embedding_dim= [12, 15, 20] #300
vocab_size= [10000]
max_length = [681]
num_epochs= [5]
param_grid = dict(optimizer=optimizer, batch_size=batch_size, embedding_dim=embedding_dim,
                  vocab_size=vocab_size, max_length=max_length, epochs=epochs)

(X_train_padded, X_test_padded, y_train, y_test) = get_data_and_split(vocab_size= 10000, max_length=681)


#vocab_size=10000, embedding_dim=300, max_length = 681, num_epochs=5

In [10]:
grid = GridSearchCV(estimator=classifer_model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train_padded, y_train) #(X, y)

TypeError: can't pickle _thread.RLock objects

In [51]:
grid = RandomizedSearchCV(estimator=classifer_model, param_distributions=param_grid, cv=4, verbose=1, n_iter=5)
grid_result = grid.fit(X_train_padded, y_train)

TypeError: can't pickle _thread.RLock objects

In [ ]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))